In [114]:
import numpy as np
import pandas as pd
import nltk
import re
import math
from eunjeon import Mecab
import networkx as nx
from tqdm import tqdm

In [115]:
data = pd.read_csv('prep_data_dotsplits.csv')
data

,본문,답변
0,정말 어려운 상황에서 너무나도 힘들어서 질문을 드립니다. 부디 자세한 설명해주시면 ...,차량의 경우 현재 점유자인 아버지를 피고로 하여 차량 인도 청구소송 및 차량 점유 ...
1,1년 전에 대포 통장이로 인해 피해를 봐서 재판 결과가 불기소 유통 지을 받았거든요...,소장 내용에 대한 검토가 필요하므로 소장을 지참하고 상담을 받으시기 바랍니다
2,연고도 다르고 일면식도 없는 5살위의 형을 일을 통해 알게 됐습니다.제가 주식투자를...,별도의 구체적 상담이 필요하나 실제 채무가 없음에도 공증을 가지고 채권자가 강제집행...
3,1 21615 이행권고결정판결 2 211523 채권압류 및 추심명령 인용 21153...,별도의 구체적 상담이 필요하나 이행 권고 결정은 기판력의 제한이 없으므로 소멸시효 ...
4,4천만 원 정도의 피해가 있고 고소를 했는데 인지사건으로 넘어가서 검찰에서 수사 중...,가해자가 위 사건으로 구속 기소되었다면 가해자를 피고로 하여 민사소송 및 강제집행으...
...,...,...
18266,제가 작년 8월쯤 승소 판결 받은 후에 압류 절차를 진행을 해야 하는데 혼자 스스로...,통장 압류 방법에 관하여 질문하셨습니다. 민사소송에서 승소를 하여 그 판결이 확정된...
18267,민사재판 패소 후 현재 등본은 부모님댁으로 되어있습니다. 나와서 따로 혼자 사는데 ...,민사소송 패소 후 부모님 집에 압류가 들어올 수 있는지와 관련하여 질문하셨습니다.민...
18268,추완항소이고 제가 피고인이예요 공시송달로 소송을 모르고 있다가 처음부터 끝까지 공시...,민사소송 항소이유를 잘못 기재한 것과 관련하여 질문하셨습니다. 부동산 임대차에 있어...
18269,작년 여름 사기 피해 금액 약 35만 원을 당하여 형사고소를 하였고 1월경 형사 고...,민사소송 소장에 소송비용 청구를 빠드린 경우와 관련하여 질 무하셨습니다. 민사소송에...


In [116]:
data.dropna(inplace=True)

In [117]:
def del_num(text):
    text = re.sub('[0-9]','',text)
    text = text.replace('ㅜㅠ', '')
    return text

In [118]:
data['본문'] = data['본문'].apply(del_num)
data['답변'] = data['답변'].apply(del_num)
data.rename(columns = {'본문':'Question', '답변':'Answer'}, inplace=True)

In [119]:
tagger = Mecab()
def similarity(a, b):
    try:
        n = len(a.intersection(b))
        return n / float(len(a) + len(b) - n)
    except:
        return 0

In [120]:
new_lines = []
dictCount={}
tag_sen = []
dictBicount={}

ratio = 0.666
max_sen = 5
threshold = 0.005

def build_dictCount(sentences):
    for sent in sentences.split('.'):
        temp_lst = []
        temp = tagger.pos(sent)
        for i in temp:
            if i[1] in ('NNG', 'NNP', 'VV', 'VA'):
                temp_lst.append(i)
        tag_sen.append(set(temp_lst))
        dictCount[len(dictCount)] = sent
    return dictCount, tag_sen

def build_dictBiCount(dictCount, tag_sen):
    for i in range(len(dictCount)):
        for j in range(i+1, len(dictCount)):
            s = similarity(tag_sen[i], tag_sen[j])
            if s < threshold:
                continue
            dictBicount[i, j] = s
    return dictBicount

def build(dictCount, dictBicount):
    graph = nx.Graph()
    graph.add_nodes_from(dictCount.keys())
    for (a, b), n in dictBicount.items():
        graph.add_edge(a, b, weight=n*1.0 + (1-1.0))
    return graph

def pagerank(graph):
    rank = nx.pagerank(graph, weight='weight')
    return rank


def new_line(rank, dictCount):
    ks = sorted(rank, key=rank.get, reverse=True)
    score = int(len(rank)*ratio)

    if score < max_sen :
        score = int(len(rank)*ratio)
    elif score >= max_sen:
        score = max_sen
    else:
        pass

    if score == 0:
        score = len(rank)

    ks = ks[:score]
    new_text = '.'.join(map(lambda k: dictCount[k], sorted(ks)))
    return new_text 

In [121]:
new_lines = []

for idx, text in tqdm(enumerate(data['Question'])):
    dictCount={}
    tag_sen = []
    dictBicount={}
    try:
        dictCount, tag_sen = build_dictCount(text)
        dictBicount = build_dictBiCount(dictCount, tag_sen)
        graph = build(dictCount, dictBicount)
        rank = pagerank(graph)
        new_text = new_line(rank, dictCount)
        new_lines.append(new_text)
    except:
        print(idx)
        break

18165it [00:30, 586.35it/s]


In [122]:
data['Question'] = new_lines

In [123]:
new_lines = []

for idx, text in tqdm(enumerate(data['Answer'])):
    dictCount={}
    tag_sen = []
    dictBicount={}
    try:
        dictCount, tag_sen = build_dictCount(text)
        dictBicount = build_dictBiCount(dictCount, tag_sen)
        graph = build(dictCount, dictBicount)
        rank = pagerank(graph)
        new_text = new_line(rank, dictCount)
        new_lines.append(new_text)
    except:
        print(idx)
        break

18165it [00:38, 466.83it/s] 


In [124]:
data['Answer'] = new_lines

In [125]:
data

,Question,Answer
0,정말 어려운 상황에서 너무나도 힘들어서 질문을 드립니다. 아버지가 제 명의로 사업자...,차량의 경우 현재 점유자인 아버지를 피고로 하여 차량 인도 청구소송 및 차량 점유 ...
1,년 전에 대포 통장이로 인해 피해를 봐서 재판 결과가 불기소 유통 지을 받았거든요....,소장 내용에 대한 검토가 필요하므로 소장을 지참하고 상담을 받으시기 바랍니다
2,제 계좌로 입금을 했다던가 그런부분 전혀없이 그 형의 계좌로 그 형의 공인인증서를 ...,별도의 구체적 상담이 필요하나 실제 채무가 없음에도 공증을 가지고 채권자가 강제집행...
3,채권시효년 소멸로 본다면 어떤 대응이 가능할지 여쭤봅니다 지인에게 연대보증 해줬는데...,별도의 구체적 상담이 필요하나 이행 권고 결정은 기판력의 제한이 없으므로 소멸시효 ...
4,그 와중에 피의자와 어떻게 연락이 닿게 되어서 갚겠다는 의사를 보였고 정말 적은 ...,가해자가 위 사건으로 구속 기소되었다면 가해자를 피고로 하여 민사소송 및 강제집행으...
...,...,...
18266,제가 작년 월쯤 승소 판결 받은 후에 압류 절차를 진행을 해야 하는데 혼자 스스로 ...,여쭤보신 것은 통장 압류 절차인 것으로 보입니다 통장 압류의 법률적 용어는 예금채...
18267,민사재판 패소 후 현재 등본은 부모님댁으로 되어있습니다,민사소송 패소 후 부모님 집에 압류가 들어올 수 있는지와 관련하여 질문하셨습니다.민...
18268,이에 원고인에게 원룸계약서 제출을 요구합니다 라고 썼거든요.거기보니 원고측에서 이미...,따라서 각 월 차임 채권은 년에 발생한 경우 년이 경과한 년 사이에 이미 시효...
18269,오늘 피고 측에서 답변서를 받고 변론 기일이 정해졌는데 이제 보니 제가 작성한 소...,민사소송 소장에 소송비용 청구를 빠드린 경우와 관련하여 질 무하셨습니다. 민사소송에...


In [132]:
data.to_csv('Textranked.csv', index=False, encoding='utf-8')

In [127]:
pre_data = pd.read_csv('prep_data_dotsplits.csv')

In [128]:
data['Question'][1000]

'사회 초년생 때 친구에게 만 원 돈을 빌려줬다가 사기당하고 못 받고 있다가 소송해서 이겨서 거의 다 받은 것 같은데 쪼금 남겨 놓고 준다고 하더니 연락을 끊었어요'

In [129]:
data['Answer'][1000]

'민사집행법 관련 조문 제조재산명시신청 ①금전의 지급을 목적으로 하는 집행권원에 기초하여 강제집행을 개시할 수 있는 채권자는 채무자의 보통재판적이 있는 곳의 법원에 채무자의 재산명시를 요구하는 신청을 할 수 있다.에게 보내야 한다 ③법원은 채무불이행자명부의 부본을 대법원규칙이 정하는 바에 따라 일정한 금융기관의 장이나 금융기관 관련 단체의 장에게 보내어 채무자에 대한 신용정보로 활용하게 할 수 있다 ④채무불이행자명부나 그 부본은 누구든지 보거나 복사할 것을 신청할 수 있다 ⑤채무불이행자명부는 인쇄물 등으로 공표되어서는 아니 된다  답변 채무자의 재산월급 예금 등을 찾아내서 그에 대해서 강제집행을 하는 방법이 있습니다.이런 경우 위의 법조에 적은 재산명시신청을 할 수 있습니다.그래도 재산을 발견하지 못하는 경우에는 채무불이행자명부 등재 신청을 하여 정신적 압박을 주는 방법이 있을 수 있습니다.재산명시신청 채무불이행자명부등재신청'

In [130]:
pre_data['본문'][1000]

'사회 초년생 때 친구에게 3만 원 돈을 빌려줬다가 사기당하고 못 받고 있다가 소송해서 이겨서 거의 다 받은 것 같은데 쪼금 남겨 놓고 준다고 하더니 연락을 끊었어요. 어떻게 해야 돌려받을 수 있을까요'

In [131]:
pre_data['답변'][1000]

'민사집행법 관련 조문 제61조재산명시신청 ①금전의 지급을 목적으로 하는 집행권원에 기초하여 강제집행을 개시할 수 있는 채권자는 채무자의 보통재판적이 있는 곳의 법원에 채무자의 재산명시를 요구하는 신청을 할 수 있다.다만 민사소송법 제213조에 따른 가집행의 선고가 붙은 판결 또는 같은 조의 준용에 따른 가집행의 선고가 붙어 집행력을 가지는 집행권원의 경우에는 그러하지 아니하다 ② 제1항의 신청에는 집행력 있는 정본과 강제집행을 개시하는데 필요한 문서를 붙여야 한다 제62조재산명시신청에 대한 재판 ①재산명시신청에 정당한 이유가 있는 때에는 법원은 채무자에게 재산상태를 명시한 재산목록을 제출하도록 명할 수 있다 ②재산명시신청에 정당한 이유가 없거나 채무자의 재산을 쉽게 찾을 수 있다고 인정한 때에는 법원은 결정으로 이를 기각하여야 한다 ③ 제1항 및 제2항의 재판은 채무자를 심문하지 아니하고 한다 ④ 제1항의 결정은 신청한 채권자 및 채무자에게 송달하여야 하고 채무자에 대한 송달에서는 결정에 따르지 아니할 경우 제68조에 규정된 제재를 받을 수 있음을 함께 고지하여야 한다 ⑤ 제4항의 규정에 따라 채무자에게 하는 송달은 민사소송법 제187조 및 제194조에 의한 방법으로는 할 수 없다 ⑥ 제1항의 결정이 채무자에게 송달되지 아니한 때에는 법원은 채권자에게 상당한 기간을 정하여 그 기간 이내에 채무자의 주소를 보정하도록 명하여야 한다 ⑦채권자가 제6항의 명령을 받고도 이를 이행하지 아니한 때에는 법원은 제1항의 결정을 취소하고 재산명시신청을 각하하여야 한다 ⑧ 제2항 및 제7항의 결정에 대하여는 즉시항고를 할 수 있다 ⑨채무자는 제1항의 결정을 송달받은 뒤 송달장소를 바꾼 때에는 그 취지를 법원에 바로 신고하여야 하며 그러한 신고를 하지 아니한 경우에는 민사소송법 제185조제2항 및 제189조의 규정을 준용한다 제64조재산명시기일의 실시 ①재산명시명령에 대하여 채무자의 이의신청이 없거나 이를 기각한 때에는 법원은 재산명시를 위한 기일을 정하여 채무자에게 출석하도록